# カテゴリデータ

## カテゴリデータの種類<a name="variation"></a>

- 名義尺度(nominal level)
 - カテゴリの順序に意味がないもの
 - 例：都道府県
- 順序尺度(ordinal scale)
 - カテゴリの順序に意味があるが、加減などの演算には意味がないもの
 - 例：アンケートの5段階評価

## カテゴリデータの変換<a name="transformation"></a>

### データ作成<a name="data"></a>

In [ ]:
import pandas as pd

df = pd.DataFrame([
    ['赤', 'S', 1000, '分類1'],
    ['緑', 'M', 1200, '分類2'],
    ['青', 'L', 1400, '分類1'],
    ['赤', 'XL', 1800, '分類2'],
    ['青', 'S', 1300, '分類1']
])
df.columns = ['色', 'サイズ', '価格', '分類']
df

### 順序尺度の変換<a name="ordinal_scale"></a>

整数値に置き換えてしまう。

In [ ]:
# マップ作成
size_mapping = {
    'S': 1,
    'M': 2,
    'L': 3,
    'XL': 4
}
# 変換
df['サイズ'] = df['サイズ'].map(size_mapping)
df

In [ ]:
# 復元
inv_size_mapping = {v: k for k, v in size_mapping.items()}
df['サイズ'] = df['サイズ'].map(inv_size_mapping)
df

In [ ]:
df['サイズ'] = df['サイズ'].map(size_mapping)

### 名義尺度の変換<a name="nominal_level"></a>

正解ラベルの変換

In [ ]:
from sklearn.preprocessing import LabelEncoder

class_encoder = LabelEncoder()
y = class_encoder.fit_transform(df['分類'].values)
y

In [ ]:
# 復元
class_encoder.inverse_transform(y)

特徴量の変換にはone-hotエンコーディングを利用(正解ラベルと同じように処理すると順序が発生してしまうので注意)

In [ ]:
# scikit-learnの機能を利用
from sklearn.preprocessing import OneHotEncoder

color_encoder = LabelEncoder()

X = df.iloc[:, :-1].values
# LabelEncoderで列0を整数値に置換
X[:, 0] = color_encoder.fit_transform(X[:, 0])
# OneHotEncoderで列0をOneHotEncoding(Sparse Matrixになるのでpandasに渡す際には通常のarrayに戻す)
one_hot = OneHotEncoder(categorical_features=[0]).fit(X)
data = one_hot.transform(X).toarray()
pd.DataFrame(data)

In [ ]:
# pandasの機能を利用
columns = list(df.columns.values)
pd.get_dummies(df[columns[:-1]])

機械学習アルゴリズムの中には、特徴同士に完全な相関があると動かないものもある。one-hotエンコーディングを行った場合、最後の特徴はその前の特徴とそのような関係になることがあるし、他の特徴から完全に再現可能なため削除することが望ましいが、scikit-learnを使う上では概ね問題なく動作する。自分で実装する場合には注意が必要。